# SQL. Сервис для чтения книг

# Описание данных и условие задачи

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато теперь они могут уделять больше времени книгам. На это обратили внимание стартаперы, которые бросились создавать новые приложения для тех, кто любит читать.<br>
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

<b>Задача</b> - проанализировать базу данных.

<i>Таблица</i> `books`

Содержит данные о книгах.<br>

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.
<br>

<i>Таблица</i> `authors`

Содержит данные об авторах.<br>

- `author_id` — идентификатор автора;
- `author` — имя автора.
<br>

<i>Таблица</i> `publishers`

Содержит данные об издательствах.<br>

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;
<br>

<i>Таблица</i> `ratings`

Содержит данные о пользовательских оценках книг.<br>

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.
<br>

<i>Таблица</i> `reviews`

Содержит данные о пользовательских обзорах на книги.<br>

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.
<br>

<img src="https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&table=block&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=1640&userId=&cache=v2" width=700/>

<br>
<b>Задания:</b><br>
- Посчитайте, сколько книг вышло после 1 января 2000 года;<br>
- Для каждой книги посчитайте количество обзоров и среднюю оценку;<br>
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;<br>
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;<br>
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.<br>

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Исследование таблиц

## books

In [3]:
query = '''
            SELECT *
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)

In [4]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''
            SELECT COUNT(*)
            FROM books
        '''
cnt_books = pd.io.sql.read_sql(query, con = engine)

In [6]:
cnt_books

,count
0,1000


Всего `1000 книг`.

## authors

In [7]:
query = '''
            SELECT *
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)

In [8]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


## publishers

In [9]:
query = '''
            SELECT *
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)

In [10]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## ratings

In [11]:
query = '''
            SELECT *
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)

In [12]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [13]:
query = '''
            SELECT COUNT(DISTINCT username)
            FROM ratings
        '''
cnt_users = pd.io.sql.read_sql(query, con = engine)

In [14]:
cnt_users

,count
0,160


Всего `160 пользователей`.

In [15]:
query = '''
            SELECT COUNT(DISTINCT book_id)
            FROM ratings
        '''
cnt_books_with_rating = pd.io.sql.read_sql(query, con = engine)

In [16]:
cnt_books_with_rating

,count
0,1000


Все книги с рейтингом.

## reviews

In [17]:
query = '''
            SELECT *
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)

In [18]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [19]:
query = '''
            SELECT COUNT(DISTINCT book_id)
            FROM reviews
        '''
cnt_books_with_reviews = pd.io.sql.read_sql(query, con = engine)

In [20]:
cnt_books_with_reviews

,count
0,994


`994` книги из `1000` с отзывами.

# SQL-запросы

## Посчитайте, сколько книг вышло после 1 января 2000 года

In [21]:
query = '''
            SELECT COUNT(*)
            FROM books
            WHERE publication_date > '2000-01-01'
        '''
books_after_1_jan_2000 = pd.io.sql.read_sql(query, con = engine)

In [22]:
books_after_1_jan_2000

,count
0,819


После `1 января 2000 года` вышло `819` книг.

## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [23]:
query = '''
            SELECT 
                book_id, 
                title, 
                COUNT(DISTINCT review_id) AS cnt_reviews,
                AVG(rating) AS avg_rating
            FROM books
            FULL JOIN reviews USING(book_id)
            JOIN ratings USING(book_id)
            GROUP BY book_id, title
            ORDER BY cnt_reviews ASC 

        '''
cnt_reviews_avg_rating_for_book = pd.io.sql.read_sql(query, con = engine)

In [24]:
cnt_reviews_avg_rating_for_book

,book_id,title,cnt_reviews,avg_rating
0,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
1,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
2,221,Essential Tales and Poems,0,4.000000
3,808,The Natural Way to Draw,0,3.000000
4,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
...,...,...,...,...
995,733,The Giver (The Giver #1),6,3.750000
996,854,The Road,6,3.772727
997,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
998,696,The Da Vinci Code (Robert Langdon #2),6,3.830508


Всего книг с отзывами и рейтингом - 994 из 1000.<br>
Всего книг с рейтингом - 1000.<br>

Книга с самым большим количеством обзоров - 7 - `Twilight (Twilight #1)`.<br>
Книги с самым низким количеством обзоров - `1.0` - 36 штук.<br>
Книги с самым высоким рейтингом - `5.0` - 43 штуки.<br>
Книга с самым низким рейтингом - 1.5 - `Harvesting the Heart`.<br>

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [25]:
query = '''
            SELECT 
                COUNT(book_id) AS summary, publisher
            FROM books
            JOIN publishers USING(publisher_id)
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY summary DESC

        '''
max_publisher_of_books = pd.io.sql.read_sql(query, con = engine)

In [26]:
max_publisher_of_books.head(10)

,summary,publisher
0,42,Penguin Books
1,31,Vintage
2,25,Grand Central Publishing
3,24,Penguin Classics
4,19,Ballantine Books
5,19,Bantam
6,17,Berkley
7,14,St. Martin's Press
8,14,Berkley Books
9,13,Delta


Самое большое количество книг более 50 страниц выпустило издательство `Penguin Books` - `42 книги`.

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [27]:
query = '''
            WITH books_and_rating AS (SELECT 
                                            book_id, 
                                            title, 
                                            author,
                                            AVG(rating) AS avg_rating,
                                            COUNT(rating_id) AS cnt_rating
                                        FROM books
                                        JOIN authors USING(author_id)
                                        JOIN ratings USING(book_id)
                                        GROUP BY book_id, author
                                        ORDER BY avg_rating DESC
                                        )
            SELECT author, AVG(avg_rating) AS avg_rating_of_books_one_author
            FROM books_and_rating
            WHERE cnt_rating >= 50
            GROUP BY author
            ORDER BY avg_rating_of_books_one_author DESC

        '''
author_max_avg_rating = pd.io.sql.read_sql(query, con = engine)

In [28]:
author_max_avg_rating

,author,avg_rating_of_books_one_author
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Автор с самой высокой средней оценкой книг - `J.K. Rowling/Mary GrandPré`. Рейтинг их книг - `4.28`.

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [29]:
query = '''
            WITH users_cnt_rating AS (SELECT
                                            username, 
                                            COUNT(rating_id) AS cnt_rating
                                        FROM ratings
                                        GROUP BY username
                                        ORDER BY cnt_rating DESC),
            users_cnt_reviews AS (SELECT
                                            username, 
                                            COUNT(review_id) AS cnt_reviews
                                        FROM reviews
                                        GROUP BY username
                                        ORDER BY cnt_reviews DESC)
            SELECT ROUND(AVG(cnt_reviews), 0)
            FROM users_cnt_rating
            JOIN users_cnt_reviews USING(username)
            WHERE cnt_rating > 50

        '''
avg_cnt_reviews_from_users_over_50_cnt_rating = pd.io.sql.read_sql(query, con = engine)

In [30]:
avg_cnt_reviews_from_users_over_50_cnt_rating

,round
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, - `24`.

# Общий вывод

Всего `1000 книг`.<br>
Всего `160 пользователей`.<br>

После `1 января 2000 года` вышло `819 книг`.<br>

Всего книг с отзывами и рейтингом - `994` из `1000`.<br>
Всего книг с рейтингом - `1000`.<br>

Книга с самым большим количеством обзоров - 7 - `Twilight (Twilight #1)`.<br>
Книги с самым низким количеством обзоров - 1 - `36 штук`.<br>
Книги с самым высоким рейтингом - `5.0` - `43 штуки`.<br>
Книга с самым низким рейтингом - `1.5` - `Harvesting the Heart`.<br>
                                                          
Самое большое количество книг `более 50 страниц` выпустило издательство `Penguin Books` - `42 книги`.<br>
                                                          
Автор с самой высокой средней оценкой книг - `J.K. Rowling/Mary GrandPré`. Рейтинг их книг - `4.28`.<br>
                                                          
Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, - `24`.<br>